In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pprint as pp

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
master_data = pd.read_csv('../input/Suicides in India 2001-2012.csv')
master_data.head()
# Any results you write to the current directory are saved as output.

,State,Year,Type_code,Type,Gender,Age_group,Total
0,A & N Islands,2001,Causes,Illness (Aids/STD),Female,0-14,0
1,A & N Islands,2001,Causes,Bankruptcy or Sudden change in Economic,Female,0-14,0
2,A & N Islands,2001,Causes,Cancellation/Non-Settlement of Marriage,Female,0-14,0
3,A & N Islands,2001,Causes,Physical Abuse (Rape/Incest Etc.),Female,0-14,0
4,A & N Islands,2001,Causes,Dowry Dispute,Female,0-14,0


In [2]:
master_data.isnull().sum()
#no null values present in the dataset

obj_type_variables = [column for column in master_data.columns if master_data[column].dtype in ['object']]
print(obj_type_variables)
for column in obj_type_variables:
    master_data[column] = master_data[column].astype('category')
master_data.info()



['State', 'Type_code', 'Type', 'Gender', 'Age_group']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237519 entries, 0 to 237518
Data columns (total 7 columns):
State        237519 non-null category
Year         237519 non-null int64
Type_code    237519 non-null category
Type         237519 non-null category
Gender       237519 non-null category
Age_group    237519 non-null category
Total        237519 non-null int64
dtypes: category(5), int64(2)
memory usage: 4.8 MB


In [3]:
#import sklearn
#from sklearn.cluster import KMeans
#kmeans=KMeans(n_clusters=5)
#kmeans.fit(master_data)


In [4]:
#labels = kmeans.predict(master_data)
#print(labels)
#centroids = kmeans.cluster_centers_
#print(centroids)

In [5]:
from kmodes.kmodes import KModes
#Cleaning data as required
suicides_data=master_data
suicides_data = suicides_data.drop(suicides_data[suicides_data.Total == 0].index)

suicides_data=suicides_data.drop(['Type'], axis=1)
suicides_data=suicides_data.drop(['Type_code'], axis=1)

suicides_data.head()


,State,Year,Gender,Age_group,Total
13,A & N Islands,2001,Female,0-14,1
20,A & N Islands,2001,Female,0-14,1
32,A & N Islands,2001,Male,0-14,1
47,A & N Islands,2001,Male,0-14,1
54,A & N Islands,2001,Female,15-29,8


In [6]:
#Selecting values with Age Group as 0-100+ only
suicides_data_selected_by_age=suicides_data.loc[suicides_data['Age_group']=="0-100+"]

suicides_data_selected_by_age = suicides_data_selected_by_age.drop(suicides_data_selected_by_age[suicides_data_selected_by_age.State.isin(
                                                                                                            ['Total (Uts)', 
                                                                                                             'Total (States)',
                                                                                                             'Total (All India)'])].index)
suicides_data_selected_by_age.head(20)
suicide_list = suicides_data_selected_by_age.values.tolist()
suicide_list = sorted(suicide_list, key=lambda x: x[0])
suicide_list = sorted(suicide_list, key=lambda x: x[2])

In [7]:
#State VS Gender Data
count = 0
final_list = list()
total_sum = 0 
for i in range(len(suicide_list)):
    if i!=0:
        if suicide_list[i-1][0] == suicide_list[i][0] and suicide_list[i-1][2] == suicide_list[i][2] and i != len(suicide_list)-1:
            total_sum+=suicide_list[i][4] 
        elif suicide_list[i-1][0] == suicide_list[i][0] and suicide_list[i-1][2] == suicide_list[i][2] and i == len(suicide_list)-1:
            total_sum+=suicide_list[i][4]
            final_list.append([suicide_list[i][0], suicide_list[i][2], total_sum])    
        else:
            final_list.append([suicide_list[i-1][0], suicide_list[i-1][2], total_sum])
            total_sum = 0
    
    else:
        total_sum+=suicide_list[i][4]          
final_list = sorted(final_list, key=lambda x: x[0])
#pp.pprint(final_list)

In [8]:
#calculating percentage and finding top 10 states vs genders based on percentage suicide
total_sum=master_data[master_data['State'] == 'Total (All India)'].Total.sum()
print("Total Number of Suicides:", total_sum)
final_list1 = list()
for row in final_list:
    if row[-1]!=((row[2]/total_sum)*100):
        row.append((row[2]/total_sum)*100)
    final_list1.append(row)
print("Top Regions w.r.t. Gender for Suicide Numbers")    
top_ten = sorted(final_list1, key = lambda x:x[3], reverse=True)[:11]
pp.pprint(top_ten)

Total Number of Suicides: 2911862
Top Regions w.r.t. Gender for Suicide Numbers
[['Maharashtra', 'Male', 243475, 8.361488284815694],
 ['Andhra Pradesh', 'Male', 216773, 7.444480542003708],
 ['Tamil Nadu', 'Male', 204211, 7.013072734902959],
 ['West Bengal', 'Male', 200726, 6.893389865316419],
 ['Karnataka', 'Male', 196764, 6.7573257249141605],
 ['Kerala', 'Male', 157763, 5.417942196436507],
 ['West Bengal', 'Female', 150760, 5.177443161798189],
 ['Tamil Nadu', 'Female', 122554, 4.208784619600792],
 ['Maharashtra', 'Female', 117220, 4.025602861674077],
 ['Andhra Pradesh', 'Female', 108786, 3.735960014588603],
 ['Madhya Pradesh', 'Male', 99319, 3.410841585212486]]


In [9]:
###Since we have identified top states --> we are planning to drill down more to find out what are the core causes of this state and how 
### can we go about it.
States=['Maharashtra','Andhra Pradesh','Tamil Nadu','West Bengal','Karnataka','Kerala','Madhya Pradesh']
suicide_by_state = master_data
suicide_by_state = suicide_by_state.drop(suicide_by_state[suicide_by_state.Total == 0].index)
#print(suicide_by_state['State'])
suicide_by_state = suicide_by_state[suicide_by_state['State'].isin(States)]

In [10]:
#States vs Causes

suicide_by_state_causes = suicide_by_state[suicide_by_state['Type_code']=="Causes"]
suicide_by_state_causes = suicide_by_state_causes.drop(['Type_code', 'Age_group'], axis=1)
suicide_by_state_causes.head(15)

,State,Year,Type,Gender,Total
6713,Andhra Pradesh,2001,Poverty,Female,4
6715,Andhra Pradesh,2001,Failure in Examination,Female,6
6716,Andhra Pradesh,2001,Family Problems,Female,20
6718,Andhra Pradesh,2001,Illness (Aids/STD),Female,4
6719,Andhra Pradesh,2001,Insanity/Mental Illness,Female,2
6720,Andhra Pradesh,2001,Cancellation/Non-Settlement of Marriage,Female,7
6721,Andhra Pradesh,2001,Causes Not known,Female,17
6725,Andhra Pradesh,2001,Physical Abuse (Rape/Incest Etc.),Female,5
6726,Andhra Pradesh,2001,Unemployment,Female,3
6727,Andhra Pradesh,2001,Love Affairs,Female,3


In [11]:
#Creating 2 different data frames 1. on the basis of year
#2.On the basis of Type

suicide_list = suicide_by_state_causes.values.tolist()
suicide_list = sorted(suicide_list, key=lambda x: x[0])
suicide_list = sorted(suicide_list, key=lambda x: x[2])


In [12]:
count = 0
final_list = list()
total_sum = 0 
for i in range(len(suicide_list)):
    if i!=0:
        if suicide_list[i-1][0] == suicide_list[i][0] and suicide_list[i-1][2] == suicide_list[i][2] and i != len(suicide_list)-1:
            total_sum+=suicide_list[i][4] 
        elif suicide_list[i-1][0] == suicide_list[i][0] and suicide_list[i-1][2] == suicide_list[i][2] and i == len(suicide_list)-1:
            total_sum+=suicide_list[i][4]
            final_list.append([suicide_list[i][0], suicide_list[i][2], total_sum])    
        else:
            final_list.append([suicide_list[i-1][0], suicide_list[i-1][2], total_sum])
            total_sum = 0
    
    else:
        total_sum+=suicide_list[i][4]  
        
        
final_list = sorted(final_list, key=lambda x: x[0])
pp.pprint(final_list)

[['Andhra Pradesh', 'Bankruptcy or Sudden change in Economic', 8112],
 ['Andhra Pradesh', 'Bankruptcy or Sudden change in Economic Status', 136],
 ['Andhra Pradesh', 'Cancellation/Non-Settlement of Marriage', 807],
 ['Andhra Pradesh', 'Cancer', 1301],
 ['Andhra Pradesh', 'Causes Not known', 12946],
 ['Andhra Pradesh', 'Death of Dear Person', 741],
 ['Andhra Pradesh', 'Divorce', 272],
 ['Andhra Pradesh', 'Dowry Dispute', 2610],
 ['Andhra Pradesh', 'Drug Abuse/Addiction', 753],
 ['Andhra Pradesh', 'Failure in Examination', 2543],
 ['Andhra Pradesh', 'Fall in Social Reputation', 959],
 ['Andhra Pradesh', 'Family Problems', 30676],
 ['Andhra Pradesh', 'Ideological Causes/Hero Worshipping', 311],
 ['Andhra Pradesh', 'Illegitimate Pregnancy', 114],
 ['Andhra Pradesh', 'Illness (Aids/STD)', 3881],
 ['Andhra Pradesh', 'Insanity/Mental Illness', 10519],
 ['Andhra Pradesh', 'Love Affairs', 3322],
 ['Andhra Pradesh', 'Not having Children (Barrenness/Impotency', 15],
 ['Andhra Pradesh', 'Not havin

In [13]:
#Calculating Totals, state wise
def calculate_total_suicide_per_state(my_list, state):
    total_sum = 0
    for i in range(len(my_list)):
        if my_list[i][0] == state:
            total_sum+=my_list[i][2]
    return total_sum

pop_dict = dict()

for state in States:
    value = calculate_total_suicide_per_state(final_list, state)
    pop_dict[state] = value
print(pop_dict)    

{'Maharashtra': 179590, 'Andhra Pradesh': 162316, 'Tamil Nadu': 163472, 'West Bengal': 160343, 'Karnataka': 146613, 'Kerala': 107755, 'Madhya Pradesh': 89658}


In [14]:
final_list1 = list()
for row in final_list:
    total_sum = pop_dict[row[0]]
    if row[-1]!=((row[2]/total_sum)*100):
        row.append((row[2]/total_sum)*100)
    final_list1.append(row)
print("Top Causes w.r.t. Selected Regions for Suicide Numbers")    
pp.pprint(final_list1)
from pandas import DataFrame
state_cause_df=DataFrame.from_records(final_list1)
#state_cause_df.head()

Top Causes w.r.t. Selected Regions for Suicide Numbers
[['Andhra Pradesh',
  'Bankruptcy or Sudden change in Economic',
  8112,
  4.997658887601961],
 ['Andhra Pradesh',
  'Bankruptcy or Sudden change in Economic Status',
  136,
  0.0837871805613741],
 ['Andhra Pradesh',
  'Cancellation/Non-Settlement of Marriage',
  807,
  0.49717834347815376],
 ['Andhra Pradesh', 'Cancer', 1301, 0.801522955223145],
 ['Andhra Pradesh', 'Causes Not known', 12946, 7.975800290790803],
 ['Andhra Pradesh', 'Death of Dear Person', 741, 0.4565169176174869],
 ['Andhra Pradesh', 'Divorce', 272, 0.1675743611227482],
 ['Andhra Pradesh', 'Dowry Dispute', 2610, 1.6079745681263709],
 ['Andhra Pradesh', 'Drug Abuse/Addiction', 753, 0.46390990413760813],
 ['Andhra Pradesh', 'Failure in Examination', 2543, 1.566697060055694],
 ['Andhra Pradesh', 'Fall in Social Reputation', 959, 0.5908228393996895],
 ['Andhra Pradesh', 'Family Problems', 30676, 18.89893787426994],
 ['Andhra Pradesh',
  'Ideological Causes/Hero Worship

In [15]:
reason_by_state = sorted(final_list1, key = lambda x:x[0])
state_wise_dict = dict()

for state in States:
    state_wise_dict[state] = list()
    for row in final_list1:
        if row[0] == state:
            state_wise_dict[state].append(row)

pp.pprint(state_wise_dict)

{'Andhra Pradesh': [['Andhra Pradesh',
                     'Bankruptcy or Sudden change in Economic',
                     8112,
                     4.997658887601961],
                    ['Andhra Pradesh',
                     'Bankruptcy or Sudden change in Economic Status',
                     136,
                     0.0837871805613741],
                    ['Andhra Pradesh',
                     'Cancellation/Non-Settlement of Marriage',
                     807,
                     0.49717834347815376],
                    ['Andhra Pradesh', 'Cancer', 1301, 0.801522955223145],
                    ['Andhra Pradesh',
                     'Causes Not known',
                     12946,
                     7.975800290790803],
                    ['Andhra Pradesh',
                     'Death of Dear Person',
                     741,
                     0.4565169176174869],
                    ['Andhra Pradesh', 'Divorce', 272, 0.1675743611227482],
                    ['Andhr

In [16]:
#Top 3 Reasons per State

def top_three_reasons(my_list):
    top_three = sorted(my_list, key = lambda x:x[3], reverse=True)[:3]
    return top_three

for state in States:
    pp.pprint(top_three_reasons(state_wise_dict[state]))

[['Maharashtra', 'Family Problems', 65297, 36.35892867086141],
 ['Maharashtra', 'Other Prolonged Illness', 33791, 18.815635614455147],
 ['Maharashtra', 'Insanity/Mental Illness', 14847, 8.267164095996437]]
[['Andhra Pradesh', 'Other Prolonged Illness', 33788, 20.816185711821387],
 ['Andhra Pradesh', 'Family Problems', 30676, 18.89893787426994],
 ['Andhra Pradesh', 'Other Causes (Please Specity)', 26979, 16.62128194386259]]
[['Tamil Nadu', 'Family Problems', 49641, 30.36666829793481],
 ['Tamil Nadu', 'Other Prolonged Illness', 31299, 19.14639815992953],
 ['Tamil Nadu', 'Other Causes (Please Specity)', 27966, 17.107516883625333]]
[['West Bengal', 'Causes Not known', 36991, 23.069918861440787],
 ['West Bengal', 'Family Problems', 35882, 22.378276569603912],
 ['West Bengal', 'Other Causes (Please Specity)', 22855, 14.253818376854618]]
[['Karnataka', 'Causes Not known', 32953, 22.47617878360036],
 ['Karnataka', 'Other Prolonged Illness', 31594, 21.549248702366093],
 ['Karnataka', 'Other Cau

In [17]:
#State vs Gender Unsupervised Learning
km = KModes(n_clusters=7, init='Huang', n_init=5, verbose=1)
#
clusters = km.fit_predict(suicides_data)

#Print the cluster centroids
print(km.cluster_centroids_)

/opt/conda/lib/python3.6/site-packages/kmodes/util/__init__.py:70: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack({tuple(row) for row in a})


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 8530, cost: 298533.0
Run 1, iteration: 2/100, moves: 0, cost: 298533.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 19704, cost: 303385.0
Run 2, iteration: 2/100, moves: 0, cost: 303385.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 4048, cost: 305885.0
Run 3, iteration: 2/100, moves: 0, cost: 305885.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 7738, cost: 310748.0
Run 4, iteration: 2/100, moves: 0, cost: 310748.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 7564, cost: 308067.0
Run 5, iteration: 2/100, moves: 0, cost: 308067.0
Best run was number 1
[['Kerala' '2002' 'Male' '30-44' '2']
 ['Tripura' '2003' 'Male

In [18]:
#State vs Gender Unsupervised Learning
km1 = KModes(n_clusters=10, init='Huang', n_init=10, verbose=1)
#

clusters = km1.fit_predict(suicide_by_state_causes)

#Print the cluster centroids
print(km1.cluster_centroids_)

Init: initializing centroids


/opt/conda/lib/python3.6/site-packages/kmodes/util/__init__.py:70: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack({tuple(row) for row in a})


Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 620, cost: 44189.0
Run 1, iteration: 2/100, moves: 0, cost: 44189.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 1910, cost: 43192.0
Run 2, iteration: 2/100, moves: 0, cost: 43192.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 889, cost: 43061.0
Run 3, iteration: 2/100, moves: 0, cost: 43061.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2115, cost: 43503.0
Run 4, iteration: 2/100, moves: 388, cost: 43503.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1196, cost: 42963.0
Run 5, iteration: 2/100, moves: 166, cost: 42963.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100, moves: 1004, cost

In [19]:
#State vs Gender Unsupervised Learning
km1 = KModes(n_clusters=10, init='Huang', n_init=10, verbose=1)
#

clusters = km1.fit_predict(state_cause_df)

#Print the cluster centroids
print(km1.cluster_centroids_)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 23, cost: 577.0
Run 1, iteration: 2/100, moves: 0, cost: 577.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 45, cost: 577.0
Run 2, iteration: 2/100, moves: 0, cost: 577.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 24, cost: 575.0
Run 3, iteration: 2/100, moves: 0, cost: 575.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 24, cost: 576.0

/opt/conda/lib/python3.6/site-packages/kmodes/util/__init__.py:70: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack({tuple(row) for row in a})



Run 4, iteration: 2/100, moves: 0, cost: 576.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 46, cost: 577.0
Run 5, iteration: 2/100, moves: 0, cost: 577.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100, moves: 42, cost: 576.0
Run 6, iteration: 2/100, moves: 17, cost: 576.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 7, iteration: 1/100, moves: 22, cost: 558.0
Run 7, iteration: 2/100, moves: 0, cost: 558.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 8, iteration: 1/100, moves: 43, cost: 576.0
Run 8, iteration: 2/100, moves: 0, cost: 576.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 9, iteration: 1/100, moves: 22, cost: 577.0
Run 9, iteration: 2/100, moves: 0, cost: 577.0
Init: initializing centroids
Init: initializing clusters
Starting iterations

In [20]:
total_sum=master_data[master_data['State'] == 'Total (All India)'].Total.sum()
print(total_sum)
master_data['Chance %']=(master_data.Total/total_sum)*100
master_data.describe()


#total_data.head()
#list(master_data.groupby(['State'],as_index=False))
#suicides_data = suicides_data.drop(suicides_data[suicides_data.Total == 0].index)

#master_data['Chance']=master_data.Total/

2911862


,Year,Total,Chance %
count,237519.000000,237519.000000,237519.000000
mean,2006.500448,55.034477,0.001890
std,3.452240,792.749038,0.027225
min,2001.000000,0.000000,0.000000
25%,2004.000000,0.000000,0.000000
50%,2007.000000,0.000000,0.000000
75%,2010.000000,6.000000,0.000206
max,2012.000000,63343.000000,2.175343


In [21]:
#Top 3 reasons for Selected States


In [22]:
#from sklearn.decomposition import FactorAnalysis
##X, _ = load_digits(return_X_y=True)
#transformer = FactorAnalysis(n_components=7, random_state=0)
#X_transformed = transformer.fit_transform(master_data[['Total','Chance %']])
#X_transformed.shape


In [23]:
'''
# Label ncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 8))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

y_train=master_data['Total']
x_train=master_data.drop(['Total'],axis=1)
#df.drop(['B', 'C'], axis=1)
classifier.fit(x_train, y_train, batch_size = 10, epochs = 100)
'''

"\n# Label ncoder\nimport keras\nfrom keras.models import Sequential\nfrom keras.layers import Dense\nclassifier = Sequential()\n\n# Adding the input layer and the first hidden layer\nclassifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 8))\n# Adding the second hidden layer\nclassifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))\n# Adding the output layer\nclassifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))\n\n# Compiling Neural Network\nclassifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])\n\ny_train=master_data['Total']\nx_train=master_data.drop(['Total'],axis=1)\n#df.drop(['B', 'C'], axis=1)\nclassifier.fit(x_train, y_train, batch_size = 10, epochs = 100)\n"